In [1]:
import numpy as np
import tensorflow as tf
import pickle

# 讀取 Muist 數字手寫資料 pickle
with open('mnist.pickle', 'rb') as file:
    (x_train, t_train), (x_test, t_test) = pickle.load(file)
    
# 圖片顯示函式
from PIL import Image

def img_show(img):
    return Image.fromarray(np.uint8(img*255))
# 顯示圖片
# try img_show(x_train[0:20])      

In [2]:
def create_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    # add one more layer and return the output of this layer
    layer_name = 'layer' + n_layer
    with tf.name_scope(layer_name):
        with tf.name_scope('Weights'):
            #Weights = tf.Variable(tf.random_normal(shape=[in_size,out_size]), name='W')
            Weights = tf.Variable(tf.random_uniform(shape=[in_size,out_size], minval=-1, maxval=1), name='W')
            tf.summary.histogram('weights', Weights)
        with tf.name_scope('biases'):
            #biases = tf.Variable(tf.random_normal(shape=[1, out_size], mean=0, stddev=0.5), name='b')
            biases = tf.Variable(tf.random_uniform(shape=[1, out_size], minval=-1, maxval=1), name='b')
            tf.summary.histogram('biases', biases)
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        tf.summary.histogram('outputs', outputs)
    return outputs

In [3]:
with tf.name_scope('inputs'):
    xs = tf.placeholder(tf.float32, [None, 784], name='x_input')
    ts = tf.placeholder(tf.float32, [None, 10], name='t_input')

layerA = create_layer(xs, 784, 30, n_layer='A', activation_function=tf.sigmoid)
layerB = create_layer(layerA, 30, 10, n_layer='B', activation_function=None) 
y = tf.nn.softmax(layerB)

# cross entropy
with tf.name_scope('loss'):
    loss = -tf.reduce_mean(tf.reduce_sum((ts * tf.log(y)), axis=1)) # 輸入為 batch 使用平均值 mean
    tf.summary.scalar('loss', loss)


#train_way = tf.train.GradientDescentOptimizer(learning_rate=0.16) # 0.9253  (learning_rate=0.16)
train_way = tf.train.MomentumOptimizer(learning_rate=0.08, momentum=0.95) # 0.9650  (learning_rate=0.08, momentum=0.95)
#train_way = tf.train.AdagradOptimizer(learning_rate=0.15) # 0.9448  (learning_rate=0.15)
#train_way = tf.train.AdamOptimizer() # 0.9513

with tf.name_scope('train'):
    train_step = train_way.minimize(loss)

with tf.name_scope('accuracy'):
    # tf.cast() 改變資料型態 np.astype change dtype
    acc = tf.cast(tf.equal(tf.argmax(y, axis=1), tf.argmax(ts, axis=1)), tf.float32)
    accuracy = tf.reduce_mean(acc, name='accuracy')
    tf.summary.scalar('accuracy', accuracy)

In [4]:
sess = tf.Session()
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter("logs/", sess.graph)
sess.run(tf.global_variables_initializer())

In [5]:
data_num = x_train.shape[0]
batch = 100
lst = np.arange(data_num)
feed_all = {xs: x_train, ts: t_train}
for i_ in range(50):
    # 隨機打亂資料排序
    np.random.shuffle(lst)
    x_train = x_train[lst]
    t_train = t_train[lst]
   
    for i in range(0, data_num, batch):  # 每 bench 100個資料
        feed_data = {xs: x_train[i:i + batch], ts: t_train[i:i + batch]}
        sess.run(train_step, feed_dict=feed_data)
        
    result = sess.run(merged, feed_dict=feed_all)
    writer.add_summary(result, i_)    
    
    # Accuracy test data
    acc_test = sess.run(accuracy, feed_dict={xs: x_test, ts: t_test})
    acc_train = sess.run(accuracy, feed_dict=feed_all)
    print('%3d  train:%.4f  test:%.4f'%(i_, acc_train, acc_test))
#sess.close()

# in win10 open powershell key:  tensorboard --logdir="logs" 

  0  train:0.9127  test:0.9158
  1  train:0.9327  test:0.9311
  2  train:0.9451  test:0.9407
  3  train:0.9517  test:0.9419
  4  train:0.9545  test:0.9488
  5  train:0.9586  test:0.9513
  6  train:0.9629  test:0.9515
  7  train:0.9641  test:0.9515
  8  train:0.9681  test:0.9547
  9  train:0.9701  test:0.9535
 10  train:0.9719  test:0.9567
 11  train:0.9733  test:0.9536
 12  train:0.9742  test:0.9547
 13  train:0.9748  test:0.9539
 14  train:0.9770  test:0.9568
 15  train:0.9778  test:0.9542
 16  train:0.9779  test:0.9570
 17  train:0.9776  test:0.9575
 18  train:0.9791  test:0.9562
 19  train:0.9811  test:0.9584
 20  train:0.9822  test:0.9581
 21  train:0.9826  test:0.9601
 22  train:0.9833  test:0.9599
 23  train:0.9826  test:0.9576
 24  train:0.9819  test:0.9570
 25  train:0.9843  test:0.9562
 26  train:0.9850  test:0.9572
 27  train:0.9854  test:0.9579
 28  train:0.9867  test:0.9587
 29  train:0.9871  test:0.9587
 30  train:0.9868  test:0.9592
 31  train:0.9842  test:0.9545
 32  tra

In [6]:
sess.close()